In [ ]:
# Author: Victor Curean and Aaron Wenteler

In [29]:
import pandas as pd
import scanpy as sc
import anndata as ad 
import pickle as pkl

In [11]:
def load_significant_perts(file_path):
    df = pd.read_csv(file_path, sep=",")
    df = df[df['significant'] == True]
    return df['Unnamed: 0'].tolist()

In [13]:
def format_norman(adata):
    adata.obs['condition'] = adata.obs['guide_ids'].replace('', 'ctrl')

In [14]:
def preprocess_adata(adata, min_gene_counts=None, min_cell_counts=None, no_highly_var=2000):
    """
    Input is an adata object has a condition column with either "ctrl" for negative controls or GENE_SYMBOL for perturbed cells
    """
    
    adata = adata.copy()

    #filter genes 
    if min_gene_counts is not None:
        sc.pp.filter_genes(adata, min_counts=min_gene_counts)

    #filter cells
    if min_cell_counts is not None:
        sc.pp.filter_cells(adata, min_counts=min_cell_counts)

    #filter only single gene perturbations and controls
    conditions_to_keep = list()
    for cond in list(adata.obs['condition']):
        if "," not in cond:
            conditions_to_keep.append(cond)
    adata = adata[adata.obs['condition'].isin(conditions_to_keep), :]

    #apply preprocessing transformation
    sc.pp.normalize_total(adata, inplace=True)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, n_top_genes=no_highly_var)
    highly_variable_genes = adata.var_names[adata.var['highly_variable']]
    adata = adata[:, highly_variable_genes]

    return adata  

In [15]:
def get_degs_for_adata(adata, covariates, group_by_col='condition', reference='ctrl', save_to_file=None):
    #returns the DEG dataframe containing statistical test results for each gene in covariates compared to the reference group
    
    sc.tl.rank_genes_groups(adata, groupby=group_by_col, reference=reference, groups=covariates, method='wilcoxon', use_raw=False, pts=True)
    deg_df_unfiltered = sc.get.rank_genes_groups_df(adata, group=None)

    #saves the file to skip computation each time
    if save_to_file is not None:
        deg_df_unfiltered.to_csv(save_to_file, sep="\t")

    return deg_df_unfiltered
    

In [20]:
def get_pert_gene_dict(deg_df, ntop_genes, adj_pval_cutoff=0.05):
    deg_df['abs_scores'] = [abs(x) for x in list(deg_df['scores'])]

    if adj_pval_cutoff is not None:
        deg_df = deg_df[deg_df['pvals_adj'] <= adj_pval_cutoff]

    pert_gene_dict = dict()
    
    for group in list(deg_df['group'].unique()):
        df = deg_df[deg_df['group'] == group]
        df = df.nlargest(ntop_genes, 'abs_scores')
        pert_gene_dict[group] = list(df['names'])

    return pert_gene_dict
        

In [ ]:
adata = ad.read_h5ad("./data/norman_2019_raw.h5ad")
perturbations_of_interest = load_significant_perts("etest_results_single_perts.csv")

format_norman(adata)

adata_pp = preprocess_adata(adata, min_gene_counts=5, min_cell_counts=None, no_highly_var=2000)
deg_df = get_degs_for_adata(adata_pp, perturbations_of_interest, group_by_col='condition', reference='ctrl', save_to_file="./deg_results.tsv")
res = get_pert_gene_dict(deg_df, 20)

In [28]:
res

{'AHR': ['ENSG00000088992',
  'ENSG00000198886',
  'ENSG00000198727',
  'ENSG00000198899',
  'ENSG00000142669',
  'ENSG00000134321',
  'ENSG00000100097',
  'ENSG00000130203',
  'ENSG00000065978',
  'ENSG00000250361',
  'ENSG00000251562',
  'ENSG00000069482',
  'ENSG00000130429',
  'ENSG00000198763',
  'ENSG00000147454',
  'ENSG00000106546',
  'ENSG00000157404',
  'ENSG00000077063',
  'ENSG00000160789',
  'ENSG00000105048'],
 'ARRDC3': ['ENSG00000279669',
  'ENSG00000167996',
  'ENSG00000268621',
  'ENSG00000187837',
  'ENSG00000245532',
  'ENSG00000099901',
  'ENSG00000115541',
  'ENSG00000008018',
  'ENSG00000117450',
  'ENSG00000147403',
  'ENSG00000069482',
  'ENSG00000100316',
  'ENSG00000164687',
  'ENSG00000126067',
  'ENSG00000145425',
  'ENSG00000096384',
  'ENSG00000136732',
  'ENSG00000041357',
  'ENSG00000110492',
  'ENSG00000069275'],
 'ATL1': ['ENSG00000158578',
  'ENSG00000130656',
  'ENSG00000167996',
  'ENSG00000250361',
  'ENSG00000196565',
  'ENSG00000205542',
  'ENSG

In [27]:
with open("../data/splits/perturb/norman/deg_pert_dict.pkl", "wb") as f:
    pkl.dump(res, f)